In [37]:
%run ../../Utils/yp_utils.py

# Initial setup

In [38]:
paper_pmid = 19220866
paper_name = 'mira_sa_correia_2009' 

In [39]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [40]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [41]:
files = ['hits_genenames.txt','hits_genenames_moderate.txt']

In [42]:
original_data_list = []
for f in files:
    original_data = pd.read_csv('raw_data/' + f, header=None, sep='\t')
    print('Original data dimensions: %d x %d' % (original_data.shape))
#     print(original_data.head())
    original_data['gene'] = original_data[0].astype(str)
    original_data['gene'] = clean_genename(original_data['gene'])
    original_data['orf'] = translate_sc(original_data['gene'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    original_data = original_data.loc[t,:]
    original_data.set_index('orf', inplace=True)
    original_data['data'] = -1
    original_data = original_data[['data']].copy()
    original_data_list.append(original_data)

Original data dimensions: 270 x 1
                 0   gene    orf
index_input                     
111          MRP16  MRP16  MRP16
Original data dimensions: 43 x 1
Empty DataFrame
Columns: [0, gene, orf]
Index: []


In [43]:
original_data = pd.concat(original_data_list, axis=1)

In [44]:
original_data.columns=['all','moderate']
original_data[original_data.isnull()] = 0
original_data['data'] = original_data['all'] - original_data['moderate'] - 1

In [45]:
original_data.index.name='orf'
original_data = original_data[['data']]

In [46]:
original_data = original_data.groupby(original_data.index).mean()

In [47]:
original_data.shape

(269, 1)

# Load & process tested strains

In [48]:
tested = pd.read_excel('raw_data/List of strains tested.xlsx', sheet_name='Tabelle2')

In [49]:
tested.head()

,ORF,slow growth?,Unnamed: 2
0,YAL068C,NaN,NaN
1,YAL067C,NaN,NaN
2,YAL066W,NaN,Note: when a strain had slow growth in control...
3,YAL065C,NaN,NaN
4,YAL062W,NaN,NaN


In [50]:
tested['orf'] = tested['ORF'].astype(str)

In [51]:
tested['orf'] = clean_orf(tested['orf'])

In [52]:
tested['orf'] = translate_sc(tested['orf'], to='orf')

In [53]:
# Make sure everything translated ok
t = looks_like_orf(tested['orf'])
print(tested.loc[~t,])

Empty DataFrame
Columns: [ORF, slow growth?, Unnamed: 2, orf]
Index: []


In [54]:
tested_orfs = tested['orf'].unique()

In [55]:
missing = [orf for orf in original_data.index.values if orf not in tested_orfs]
missing

['YBR132C', 'YGL188W-A', 'YPR175W-A']

In [56]:
tested_orfs = list(tested_orfs) + missing

In [57]:
original_data = original_data.reindex(index=tested_orfs, fill_value=0)

# Prepare the final dataset

In [58]:
data = original_data.copy()

In [59]:
dataset_ids = [157]
datasets = datasets.reindex(index=dataset_ids)

In [60]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [61]:
data.head()

dataset_id,157
data_type,value
orf,
YAL068C,0.0
YAL067C,0.0
YAL066W,0.0
YAL065C,0.0
YAL062W,0.0


## Subset to the genes currently in SGD

In [62]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [63]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,157
,data_type,value
gene_id,orf,
1869,YAL068C,0.0
61,YAL067C,0.0
60,YAL066W,0.0
1727,YAL065C,0.0
57,YAL062W,0.0


# Normalize

In [64]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [65]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [66]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id        157       
data_type       value valuez
gene_id orf                 
1869    YAL068C   0.0    0.0
61      YAL067C   0.0    0.0
60      YAL066W   0.0    0.0
1727    YAL065C   0.0    0.0
57      YAL062W   0.0    0.0

# Print out

In [67]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [71]:
# from IO.save_data_to_db3 import *

In [72]:
# save_data_to_db(data_all, paper_pmid)